# BURGER_AGENT

### AGENT.py

In [1]:
%pip install crewai


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import Literal
from pydantic import BaseModel
import uuid
from crewai import Agent, Crew, LLM, Task, Process
from crewai.tools import tool
from dotenv import load_dotenv
import litellm
# litellm.vertex_project = os.getenv("GCLOUD_PROJECT_ID")
# litellm.vertex_location = os.getenv("GCLOUD_LOCATION")

In [3]:
class ResponseFormat(BaseModel):
    """Respond to the user in this format."""

    status: Literal["input_required", "completed", "error"] = "input_required"
    message: str


class OrderItem(BaseModel):
    name: str
    quantity: int
    price: int


class Order(BaseModel):
    order_id: str
    status: str
    order_items: list[OrderItem]


@tool("create_order")
def create_burger_order(order_items: list[OrderItem]) -> str:
    """
    Creates a new burger order with the given order items.

    Args:
        order_items: List of order items to be added to the order.

    Returns:
        str: A message indicating that the order has been created.
    """
    try:
        order_id = str(uuid.uuid4())
        order = Order(order_id=order_id, status="created", order_items=order_items)
        print("===")
        print(f"order created: {order}")
        print("===")
    except Exception as e:
        print(f"Error creating order: {e}")
        return f"Error creating order: {e}"
    return f"Order {order.model_dump()} has been created"

In [4]:
class BurgerSellerAgent:
    TaskInstruction = """
# INSTRUCTIONS

You are a specialized assistant for a burger store.
Your sole purpose is to answer questions about what is available on burger menu and price also handle order creation.
If the user asks about anything other than burger menu or order creation, politely state that you cannot help with that topic and can only assist with burger menu and order creation.
Do not attempt to answer unrelated questions or use tools for other purposes.

# CONTEXT

Received user query: {user_prompt}
Session ID: {session_id}

Provided below is the available burger menu and it's related price:
- Classic Cheeseburger: IDR 85K
- Double Cheeseburger: IDR 110K
- Spicy Chicken Burger: IDR 80K
- Spicy Cajun Burger: IDR 85K

# RULES

- If user want to do something, you will be following this order:
    1. Always ensure the user already confirmed the order and total price. This confirmation may already given in the user query.
    2. Use `create_burger_order` tool to create the order
    3. Finally, always provide response to the user about the detailed ordered items, price breakdown and total, and order ID
    
- Set response status to input_required if asking for user order confirmation.
- Set response status to error if there is an error while processing the request.
- Set response status to completed if the request is complete.
- DO NOT make up menu or price, Always rely on the provided menu given to you as context.
"""
    SUPPORTED_CONTENT_TYPES = ["text", "text/plain"]

    def invoke(self, query, sessionId) -> str:
        burger_agent = Agent(
            role="Burger Seller Agent",
            goal=(
                "Help user to understand what is available on burger menu and price also handle order creation."
            ),
            backstory=("You are an expert and helpful burger seller agent."),
            verbose=False,
            allow_delegation=False,
            tools=[create_burger_order],
            llm=LLM(
                model="hosted_vllm/meta-llama/Llama-3.1-8B-Instruct", # os.getenv("VLLM_MODEL"), #VLLM_MODEL
                api_base="http://localhost:8088/v1" # os.getenv("OPENAI_API_BASE") # OPENAI_API_BASE
                )
        )

        agent_task = Task(
            description=self.TaskInstruction,
            output_pydantic=ResponseFormat,
            agent=burger_agent,
            expected_output=(
                "A JSON object with 'status' and 'message' fields."
                "Set response status to input_required if asking for user order confirmation."
                "Set response status to error if there is an error while processing the request."
                "Set response status to completed if the request is complete."
            ),
        )

        crew = Crew(
            tasks=[agent_task],
            agents=[burger_agent],
            verbose=False,
            process=Process.sequential,
        )

        inputs = {"user_prompt": query, "session_id": sessionId}
        response = crew.kickoff(inputs)
        return self.get_agent_response(response)

    def get_agent_response(self, response):
        response_object = response.pydantic
        if response_object and isinstance(response_object, ResponseFormat):
            if response_object.status == "input_required":
                return {
                    "is_task_complete": False,
                    "require_user_input": True,
                    "content": response_object.message,
                }
            elif response_object.status == "error":
                return {
                    "is_task_complete": False,
                    "require_user_input": True,
                    "content": response_object.message,
                }
            elif response_object.status == "completed":
                return {
                    "is_task_complete": True,
                    "require_user_input": False,
                    "content": response_object.message,
                }

        return {
            "is_task_complete": False,
            "require_user_input": True,
            "content": "We are unable to process your request at the moment. Please try again.",
        }

### MAIN.py

In [5]:
import sys
import os

project_root = os.path.abspath("remote_seller_agents/burger_agent")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(project_root)

/home/pbaskara/trial/Jupyter/orig/tut_branhc/purchasing-concierge-intro-a2a-codelab-starter/remote_seller_agents/burger_agent


In [6]:
# """
# Copyright 2025 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# """

from a2a_server.server import A2AServer
from a2a_types import AgentCard, AgentCapabilities, AgentSkill, AgentAuthentication
from a2a_server.push_notification_auth import PushNotificationSenderAuth
from task_manager import AgentTaskManager
from agent import BurgerSellerAgent
import click
import logging
from dotenv import load_dotenv
import os

load_dotenv()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import threading
import time
import os
import logging

AUTH_USERNAME="burgeruser123"
AUTH_PASSWORD="burgerpass123"

In [7]:
def main(host, port):
    """Starts the Burger Seller Agent server."""
    try:
        capabilities = AgentCapabilities(pushNotifications=True)
        skill = AgentSkill(
            id="create_burger_order",
            name="Burger Order Creation Tool",
            description="Helps with creating burger orders",
            tags=["burger order creation"],
            examples=["I want to order 2 classic cheeseburgers"],
        )
        agent_card = AgentCard(
            name="burger_seller_agent",
            description="Helps with creating burger orders",
            # The URL provided here is for the sake of demo,
            # in production you should use a proper domain name
            url=f"http://{host}:{port}/",
            version="1.0.0",
            authentication=AgentAuthentication(schemes=["Basic"]),
            defaultInputModes=BurgerSellerAgent.SUPPORTED_CONTENT_TYPES,
            defaultOutputModes=BurgerSellerAgent.SUPPORTED_CONTENT_TYPES,
            capabilities=capabilities,
            skills=[skill],
        )


        notification_sender_auth = PushNotificationSenderAuth()
        notification_sender_auth.generate_jwk()
        server = A2AServer(
            agent_card=agent_card,
            task_manager=AgentTaskManager(
                agent=BurgerSellerAgent(),
                notification_sender_auth=notification_sender_auth,
            ),
            host=host,
            port=port,
            auth_username=AUTH_USERNAME,
            auth_password=AUTH_PASSWORD,
        )

        server.app.add_route(
            "/.well-known/jwks.json",
            notification_sender_auth.handle_jwks_endpoint,
            methods=["GET"],
        )

        logger.info(f"Starting server on {host}:{port}")
        server.start()
    except Exception as e:
        logger.error(f"An error occurred during server startup: {e}")
        exit(1)

In [8]:
# --- Global variable to hold the server thread reference ---
# This allows you to stop it later from another cell if needed
global server_thread
server_thread = None

# --- Main execution in the Jupyter cell ---
if server_thread is not None and server_thread.is_alive():
    print("Server is already running.")
else:
    # Define host and port
    server_host = "0.0.0.0"
    server_port = 10003

    # Create and start the thread
    server_thread = threading.Thread(target=main, args=(server_host, server_port))
    server_thread.daemon = True # Allows the main program to exit even if the thread is still running
    server_thread.start()

    print(f"Server thread started. Waiting a moment for server to initialize on http://{server_host}:{server_port}")
    time.sleep(5) # Give it a few seconds to boot up

    # You can now proceed with other cells, or client code in this cell
    # Example client interaction (assuming your server exposes an endpoint)
    # import requests
    # try:
    #     response = requests.get(f"http://127.0.0.1:{server_port}/") # Or your specific endpoint
    #     response.raise_for_status()
    #     print(f"Successfully connected to server. Status: {response.status_code}")
    #     # print("Server root response:", response.text)
    # except requests.exceptions.ConnectionError:
    #     print(f"Error: Could not connect to the server at http://127.0.0.1:{server_port}/. Is it running?")
    # except Exception as e:
    #     print(f"An error occurred during client connection: {e}")

INFO:__main__:Starting server on 0.0.0.0:10003
INFO:     Started server process [911039]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:10003 (Press CTRL+C to quit)


Server thread started. Waiting a moment for server to initialize on http://0.0.0.0:10003


INFO:     127.0.0.1:45056 - "GET /.well-known/agent.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:42972 - "GET /.well-known/agent.json HTTP/1.1" 200 OK


INFO:a2a_server.task_manager:Upserting task 9d216e45-59dc-4a99-b14a-c65bd469bbf6
INFO:task_manager:No push notification info found for task 9d216e45-59dc-4a99-b14a-c65bd469bbf6
09:50:38 - LiteLLM:INFO: utils.py:3224 - 
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm


jsonrpc='2.0' id='7912405a081a4fc7bf8252dbe546bfca' method='tasks/send' params=TaskSendParams(id='9d216e45-59dc-4a99-b14a-c65bd469bbf6', sessionId='5fdeaeca-9a07-4693-a942-9fa4ca0398ad', message=Message(role='user', parts=[TextPart(type='text', text='User is inquiring about burgers. Please assist with creating a burger order.', metadata=None)], metadata={'conversation_id': '5fdeaeca-9a07-4693-a942-9fa4ca0398ad', 'message_id': 'dcd2af92-4d3e-4ad9-9d33-5f8057ff8ece'}), acceptedOutputModes=['text', 'text/plain'], pushNotification=None, historyLength=None, metadata={'conversation_id': '5fdeaeca-9a07-4693-a942-9fa4ca0398ad'})


...

INFO:httpx:HTTP Request: POST http://localhost:8088/v1/chat/completions "HTTP/1.1 200 OK"
09:50:42 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:task_manager:No push notification info found for task 9d216e45-59dc-4a99-b14a-c65bd469bbf6
INFO:a2a_server.task_manager:Upserting task 2a22fcfd-064b-4e2f-a955-0dbe5f5ee813
INFO:task_manager:No push notification info found for task 2a22fcfd-064b-4e2f-a955-0dbe5f5ee813
09:50:48 - LiteLLM:INFO: utils.py:3224 - 
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm


jsonrpc='2.0' id='270b3f2981d846e3a28767d889d3c878' method='tasks/send' params=TaskSendParams(id='2a22fcfd-064b-4e2f-a955-0dbe5f5ee813', sessionId='5fdeaeca-9a07-4693-a942-9fa4ca0398ad', message=Message(role='user', parts=[TextPart(type='text', text='Please confirm the order with items Classic Cheeseburger and Spicy Chicken Burger with a total cost of 165,000', metadata=None)], metadata={'conversation_id': '5fdeaeca-9a07-4693-a942-9fa4ca0398ad', 'message_id': 'ad549a5b-9d7d-4fe1-8596-3a5f99ed30b4'}), acceptedOutputModes=['text', 'text/plain'], pushNotification=None, historyLength=None, metadata={'conversation_id': '5fdeaeca-9a07-4693-a942-9fa4ca0398ad'})


INFO:httpx:HTTP Request: POST http://localhost:8088/v1/chat/completions "HTTP/1.1 200 OK"
09:50:48 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
09:50:48 - LiteLLM:INFO: utils.py:3224 - 
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm
INFO:httpx:HTTP Request: POST http://localhost:8088/v1/chat/completions "HTTP/1.1 200 OK"
09:50:49 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
09:50:49 - LiteLLM:INFO: utils.py:3224 - 
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm
INFO:httpx:HTTP Request: POST http://localhost:8088/v1/chat/comp

===
order created: order_id='e319c3dd-5baa-4e87-b30c-107f94ab0994' status='created' order_items=[OrderItem(name='Classic Cheeseburger', quantity=1, price=85000), OrderItem(name='Spicy Chicken Burger', quantity=1, price=80000)]
===


INFO:httpx:HTTP Request: POST http://localhost:8088/v1/chat/completions "HTTP/1.1 200 OK"
09:50:50 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:task_manager:No push notification info found for task 2a22fcfd-064b-4e2f-a955-0dbe5f5ee813


In [9]:
import os

# OPENAI_API_BASE="http://localhost:8088/v1" # vLLM serve URL (we used port 8088 here)
# VLLM_MODEL="hosted_vllm/meta-llama/Llama-3.1-8B-Instruct"
# Set these to the correct values for your setup
os.environ["VLLM_MODEL"] = "hosted_vllm/meta-llama/Llama-3.1-8B-Instruct"
os.environ["OPENAI_API_BASE"] = "http://localhost:8088/v1"

In [10]:
agent = BurgerSellerAgent()
print(agent) 
result = agent.invoke("1 classic cheeseburger pls", "default_session")
print(result)

09:49:40 - LiteLLM:INFO: utils.py:3224 - 
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm


INFO:httpx:HTTP Request: POST http://localhost:8088/v1/chat/completions "HTTP/1.1 200 OK"
09:49:40 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
09:49:40 - LiteLLM:INFO: utils.py:3224 - 
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/Llama-3.1-8B-Instruct; provider = hosted_vllm
INFO:httpx:HTTP Request: POST http://localhost:8088/v1/chat/completions "HTTP/1.1 200 OK"
09:49:41 - LiteLLM:INFO: utils.py:1236 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


{'is_task_complete': True, 'require_user_input': False, 'content': 'Your order has been created. You have ordered 1 Classic Cheeseburger for IDR 85,000. Your order ID is #001.'}


# PIZZA_AGENT

# ROOT_AGENT

### AGENT.py

In [11]:
import sys
import os

# Adjust path to point to the folder containing a2a_server
project_root = os.path.abspath("purchasing_concierge/")
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
# print("Project root added to sys.path:", project_root)

In [12]:
os.environ["OLLAMA_MODEL"] = "ollama_chat/llama3.1:latest"
os.environ["OLLAMA_BASE_URL"] = "http://localhost:11434"


# os.environ["PIZZA_SELLER_AGENT_AUTH"] = "pizza123"
# os.environ["PIZZA_SELLER_AGENT_URL"] = "http://localhost:10004"
os.environ["BURGER_SELLER_AGENT_AUTH"] = "burgeruser123:burgerpass123"
os.environ["BURGER_SELLER_AGENT_URL"] = "http://localhost:10003"
# GOOGLE_GENAI_USE_VERTEXAI=TRUE
# GOOGLE_CLOUD_PROJECT={your-project-id}
# GOOGLE_CLOUD_LOCATION=us-central1
# OLLAMA_MODEL="ollama_chat/llama3.1:latest"
# OLLAMA_BASE_URL="http://localhost:11434"

In [13]:
from purchasing_agent import PurchasingAgent
from dotenv import load_dotenv
import os

load_dotenv()

root_agent = PurchasingAgent(
    remote_agent_addresses=[
        # os.getenv("PIZZA_SELLER_AGENT_URL", "http://localhost:10004"),
        os.getenv("BURGER_SELLER_AGENT_URL", "http://localhost:10003"),
    ]
).create_agent()


# # Send a sample task
# result = root_agent.run("I want to order two burgers")
# print("Response:", result)


INFO:google.adk.models.registry:Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/endpoints\/.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/publishers\/google\/models\/gemini.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google.adk.models.registry:Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/endpoints\/.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google.adk.models.registry:Updating LLM class fo

Found agent card: {'name': 'burger_seller_agent', 'description': 'Helps with creating burger orders', 'url': 'http://0.0.0.0:10003/', 'provider': None, 'version': '1.0.0', 'documentationUrl': None, 'capabilities': {'streaming': False, 'pushNotifications': True, 'stateTransitionHistory': False}, 'authentication': {'schemes': ['Basic'], 'credentials': None}, 'defaultInputModes': ['text', 'text/plain'], 'defaultOutputModes': ['text', 'text/plain'], 'skills': [{'id': 'create_burger_order', 'name': 'Burger Order Creation Tool', 'description': 'Helps with creating burger orders', 'tags': ['burger order creation'], 'examples': ['I want to order 2 classic cheeseburgers'], 'inputModes': None, 'outputModes': None}]}


### REMOTE_AGENT_CONNECTION

In [14]:
"""
Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

from typing import Callable
import uuid
from a2a_types import (
    AgentCard,
    Task,
    TaskSendParams,
    TaskStatusUpdateEvent,
    TaskArtifactUpdateEvent,
)
from a2a_client.client import A2AClient
from dotenv import load_dotenv
import os

load_dotenv()

TaskCallbackArg = Task | TaskStatusUpdateEvent | TaskArtifactUpdateEvent
TaskUpdateCallback = Callable[[TaskCallbackArg, AgentCard], Task]

KNOWN_AUTH = {
    # "pizza_seller_agent": os.getenv("PIZZA_SELLER_AGENT_AUTH", "api_key"),
    "burger_seller_agent": os.getenv("BURGER_SELLER_AGENT_AUTH", "user:pass"),
}


class RemoteAgentConnections:
    """A class to hold the connections to the remote agents."""

    def __init__(self, agent_card: AgentCard, agent_url: str):
        auth = KNOWN_AUTH.get(agent_card.name, None)
        self.agent_client = A2AClient(agent_card, auth=auth, agent_url=agent_url)
        self.card = agent_card

        self.conversation_name = None
        self.conversation = None
        self.pending_tasks = set()

    def get_agent(self) -> AgentCard:
        return self.card

    async def send_task(
        self,
        request: TaskSendParams,
        task_callback: TaskUpdateCallback | None,
    ) -> Task | None:
        response = await self.agent_client.send_task(request.model_dump())
        merge_metadata(response.result, request)
        # For task status updates, we need to propagate metadata and provide
        # a unique message id.
        if (
            hasattr(response.result, "status")
            and hasattr(response.result.status, "message")
            and response.result.status.message
        ):
            merge_metadata(response.result.status.message, request.message)
            m = response.result.status.message
            if not m.metadata:
                m.metadata = {}
            if "message_id" in m.metadata:
                m.metadata["last_message_id"] = m.metadata["message_id"]
            m.metadata["message_id"] = str(uuid.uuid4())

        if task_callback:
            task_callback(response.result, self.card)
        return response.result


def merge_metadata(target, source):
    if not hasattr(target, "metadata") or not hasattr(source, "metadata"):
        return
    if target.metadata and source.metadata:
        target.metadata.update(source.metadata)
    elif source.metadata:
        target.metadata = dict(**source.metadata)


### PURCHASING_AGENT - ROOT

In [15]:
"""
Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

import json
import uuid
from typing import List
import httpx

from dotenv import load_dotenv
import os
load_dotenv()

# Custom by Shailen
from google.adk.models.lite_llm import LiteLlm 

from google.adk import Agent
from google.adk.agents.readonly_context import ReadonlyContext
from google.adk.agents.callback_context import CallbackContext
from google.adk.tools.tool_context import ToolContext
from remote_agent_connection import RemoteAgentConnections, TaskUpdateCallback
from a2a_client.card_resolver import A2ACardResolver
from a2a_types import (
    AgentCard,
    Message,
    TaskState,
    Task,
    TaskSendParams,
    TextPart,
    Part,
)


class PurchasingAgent:
    """The purchasing agent.

    This is the agent responsible for choosing which remote seller agents to send
    tasks to and coordinate their work.
    """

    def __init__(
        self,
        remote_agent_addresses: List[str],
        task_callback: TaskUpdateCallback | None = None,
    ):
        self.task_callback = task_callback
        self.remote_agent_connections: dict[str, RemoteAgentConnections] = {}
        self.cards: dict[str, AgentCard] = {}
        for address in remote_agent_addresses:
            card_resolver = A2ACardResolver(address)
            try:
                card = card_resolver.get_agent_card()
                # The URL accessed here should be the same as the one provided in the agent card
                # However, in this demo we are using the URL provided in the key arguments
                remote_connection = RemoteAgentConnections(
                    agent_card=card, agent_url=address
                )
                self.remote_agent_connections[card.name] = remote_connection
                self.cards[card.name] = card
            except httpx.ConnectError:
                print(f"ERROR: Failed to get agent card from : {address}")
        agent_info = []
        for ra in self.list_remote_agents():
            agent_info.append(json.dumps(ra))
        self.agents = "\n".join(agent_info)

    def create_agent(self) -> Agent:
        return Agent(
            model=LiteLlm(model=os.getenv("OLLAMA_MODEL")), #"gemini-2.0-flash-001",
            name="purchasing_agent",
            instruction=self.root_instruction,
            before_model_callback=self.before_model_callback,
            description=(
                "This purchasing agent orchestrates the decomposition of the user purchase request into"
                " tasks that can be performed by the seller agents."
            ),
            tools=[
                self.send_task,
            ],
        )

    def root_instruction(self, context: ReadonlyContext) -> str:
        current_agent = self.check_active_agent(context)
        return f"""You are an expert purchasing delegator that can delegate the user product inquiry and purchase request to the
appropriate seller remote agents.

Execution:
- For actionable tasks, you can use `send_task` to assign tasks to remote agents to perform.
- When the remote agent is repeatedly asking for user confirmation, assume that the remote agent doesn't have access to user's conversation context. 
    So improve the task description to include all the necessary information related to that agent
- Never ask user permission when you want to connect with remote agents. If you need to make connection with multiple remote agents, directly
    connect with them without asking user permission or asking user preference
- Always show the detailed response information from the seller agent and propagate it properly to the user. 
- If the remote seller is asking for confirmation, rely the confirmation question to the user if the user haven't do so. 
- If the user already confirmed the related order in the past conversation history, you can confirm on behalf of the user
- Do not give irrelevant context to remote seller agent. For example, ordered pizza item is not relevant for the burger seller agent
- Never ask order confirmation to the remote seller agent 

Please rely on tools to address the request, and don't make up the response. If you are not sure, please ask the user for more details.
Focus on the most recent parts of the conversation primarily.

If there is an active agent, send the request to that agent with the update task tool.

Agents:
{self.agents}

Current active seller agent: {current_agent["active_agent"]}
"""

    def check_active_agent(self, context: ReadonlyContext):
        state = context.state
        if (
            "session_id" in state
            and "session_active" in state
            and state["session_active"]
            and "active_agent" in state
        ):
            return {"active_agent": f"{state['active_agent']}"}
        return {"active_agent": "None"}

    def before_model_callback(self, callback_context: CallbackContext, llm_request):
        state = callback_context.state
        if "session_active" not in state or not state["session_active"]:
            if "session_id" not in state:
                state["session_id"] = str(uuid.uuid4())
            state["session_active"] = True

    def list_remote_agents(self):
        """List the available remote agents you can use to delegate the task."""
        if not self.remote_agent_connections:
            return []

        remote_agent_info = []
        for card in self.cards.values():
            print(f"Found agent card: {card.model_dump()}")
            print("=" * 100)
            remote_agent_info.append(
                {"name": card.name, "description": card.description}
            )
        return remote_agent_info

    async def send_task(self, agent_name: str, task: str, tool_context: ToolContext):
        """Sends a task to remote seller agent

        This will send a message to the remote agent named agent_name.

        Args:
            agent_name: The name of the agent to send the task to.
            task: The comprehensive conversation context summary
                and goal to be achieved regarding user inquiry and purchase request.
            tool_context: The tool context this method runs in.

        Yields:
            A dictionary of JSON data.
        """
        if agent_name not in self.remote_agent_connections:
            raise ValueError(f"Agent {agent_name} not found")
        state = tool_context.state
        state["active_agent"] = agent_name
        client = self.remote_agent_connections[agent_name]
        if not client:
            raise ValueError(f"Client not available for {agent_name}")
        if "task_id" in state:
            taskId = state["task_id"]
        else:
            taskId = str(uuid.uuid4())
        sessionId = state["session_id"]
        task: Task
        messageId = ""
        metadata = {}
        if "input_message_metadata" in state:
            metadata.update(**state["input_message_metadata"])
            if "message_id" in state["input_message_metadata"]:
                messageId = state["input_message_metadata"]["message_id"]
        if not messageId:
            messageId = str(uuid.uuid4())
        metadata.update(**{"conversation_id": sessionId, "message_id": messageId})
        request: TaskSendParams = TaskSendParams(
            id=taskId,
            sessionId=sessionId,
            message=Message(
                role="user",
                parts=[TextPart(text=task)],
                metadata=metadata,
            ),
            acceptedOutputModes=["text", "text/plain"],
            # pushNotification=None,
            metadata={"conversation_id": sessionId},
        )
        task = await client.send_task(request, self.task_callback)
        # Assume completion unless a state returns that isn't complete
        state["session_active"] = task.status.state not in [
            TaskState.COMPLETED,
            TaskState.CANCELED,
            TaskState.FAILED,
            TaskState.UNKNOWN,
        ]
        if task.status.state == TaskState.INPUT_REQUIRED:
            # Force user input back
            tool_context.actions.escalate = True
        elif task.status.state == TaskState.COMPLETED:
            # Reset active agent is task is completed
            state["active_agent"] = "None"

        response = []
        if task.status.message:
            # Assume the information is in the task message.
            response.extend(convert_parts(task.status.message.parts, tool_context))
        if task.artifacts:
            for artifact in task.artifacts:
                response.extend(convert_parts(artifact.parts, tool_context))
        return response


def convert_parts(parts: list[Part], tool_context: ToolContext):
    rval = []
    for p in parts:
        rval.append(convert_part(p, tool_context))
    return rval


def convert_part(part: Part, tool_context: ToolContext):
    # Currently only support text parts
    if part.type == "text":
        return part.text

    return f"Unknown type: {part.type}"


### Run the purchasing concierge agent with the UI

In [ ]:
"""
Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

import gradio as gr
from typing import List, Dict, Any
from purchasing_concierge.agent import root_agent as purchasing_agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.events import Event
from typing import AsyncIterator
from google.genai import types
from pprint import pformat

APP_NAME = "purchasing_concierge_app"
USER_ID = "default_user"
SESSION_ID = "default_session"
SESSION_SERVICE = InMemorySessionService()
PURCHASING_AGENT_RUNNER = Runner(
    agent=purchasing_agent,  # The agent we want to run
    app_name=APP_NAME,  # Associates runs with our app
    session_service=SESSION_SERVICE,  # Uses our session manager
)
SESSION_SERVICE.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)


async def get_response_from_agent(
    message: str,
    history: List[Dict[str, Any]],
) -> str:
    """Send the message to the backend and get a response.

    Args:
        message: Text content of the message.
        history: List of previous message dictionaries in the conversation.

    Returns:
        Text response from the backend service.
    """
    # try:
    events_iterator: AsyncIterator[Event] = PURCHASING_AGENT_RUNNER.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=types.Content(role="user", parts=[types.Part(text=message)]),
    )

    responses = []
    async for event in events_iterator:  # event has type Event
        if event.content.parts:
            for part in event.content.parts:
                if part.function_call:
                    formatted_call = f"```python\n{pformat(part.function_call.model_dump(), indent=2, width=80)}\n```"
                    responses.append(
                        gr.ChatMessage(
                            role="assistant",
                            content=f"{part.function_call.name}:\n{formatted_call}",
                            metadata={"title": "🛠️ Tool Call"},
                        )
                    )
                elif part.function_response:
                    formatted_response = f"```python\n{pformat(part.function_response.model_dump(), indent=2, width=80)}\n```"

                    responses.append(
                        gr.ChatMessage(
                            role="assistant",
                            content=formatted_response,
                            metadata={"title": "⚡ Tool Response"},
                        )
                    )

        # Key Concept: is_final_response() marks the concluding message for the turn
        if event.is_final_response():
            if event.content and event.content.parts:
                # Extract text from the first part
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate:
                # Handle potential errors/escalations
                final_response_text = (
                    f"Agent escalated: {event.error_message or 'No specific message.'}"
                )
            responses.append(
                gr.ChatMessage(role="assistant", content=final_response_text)
            )
            yield responses
            break  # Stop processing events once the final response is found

        yield responses
    # except Exception as e:
    #     yield [
    #         gr.ChatMessage(
    #             role="assistant",
    #             content=f"Error communicating with agent: {str(e)}",
    #         )
    #     ]


if __name__ == "__main__":
    demo = gr.ChatInterface(
        get_response_from_agent,
        title="Purchasing Concierge",
        description="This assistant can help you to purchase food from remote sellers.",
        type="messages",
    )

    demo.launch(
        server_name="0.0.0.0",
        server_port=8080,
    )

/home/pbaskara/trial/Jupyter/orig/tut_branhc/purchasing-concierge-intro-a2a-codelab-starter/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:httpx:HTTP Request: GET http://localhost:10003/.well-known/agent.json "HTTP/1.1 200 OK"


Found agent card: {'name': 'burger_seller_agent', 'description': 'Helps with creating burger orders', 'url': 'http://0.0.0.0:10003/', 'provider': None, 'version': '1.0.0', 'documentationUrl': None, 'capabilities': {'streaming': False, 'pushNotifications': True, 'stateTransitionHistory': False}, 'authentication': {'schemes': ['Basic'], 'credentials': None}, 'defaultInputModes': ['text', 'text/plain'], 'defaultOutputModes': ['text', 'text/plain'], 'skills': [{'id': 'create_burger_order', 'name': 'Burger Order Creation Tool', 'description': 'Helps with creating burger orders', 'tags': ['burger order creation'], 'examples': ['I want to order 2 classic cheeseburgers'], 'inputModes': None, 'outputModes': None}]}


INFO:httpx:HTTP Request: GET http://localhost:8080/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://localhost:8080/ "HTTP/1.1 200 OK"


* Running on local URL:  http://0.0.0.0:8080
* To create a public link, set `share=True` in `launch()`.


...

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/home/pbaskara/trial/Jupyter/orig/tut_branhc/purchasing-concierge-intro-a2a-codelab-starter/.venv/lib/python3.12/site-packages/opentelemetry/trace/__init__.py", line 589, in use_span
    yield span
  File "/home/pbaskara/trial/Jupyter/orig/tut_branhc/purchasing-concierge-intro-a2a-codelab-starter/.venv/lib/python3.12/site-packages/opentelemetry/sdk/trace/__init__.py", line 1105, in start_as_current_span
    yield span
  File "/home/pbaskara/trial/Jupyter/orig/tut_branhc/purchasing-concierge-intro-a2a-codelab-starter/.venv/lib/python3.12/site-packages/google/adk/runners.py", lin

Send Remote Agent Task Request: {'jsonrpc': '2.0', 'id': '270b3f2981d846e3a28767d889d3c878', 'method': 'tasks/send', 'params': {'id': '2a22fcfd-064b-4e2f-a955-0dbe5f5ee813', 'sessionId': '5fdeaeca-9a07-4693-a942-9fa4ca0398ad', 'message': {'role': 'user', 'parts': [{'type': 'text', 'text': 'Please confirm the order with items Classic Cheeseburger and Spicy Chicken Burger with a total cost of 165,000', 'metadata': None}], 'metadata': {'conversation_id': '5fdeaeca-9a07-4693-a942-9fa4ca0398ad', 'message_id': 'ad549a5b-9d7d-4fe1-8596-3a5f99ed30b4'}}, 'acceptedOutputModes': ['text', 'text/plain'], 'pushNotification': None, 'historyLength': None, 'metadata': {'conversation_id': '5fdeaeca-9a07-4693-a942-9fa4ca0398ad'}}}


INFO:httpx:HTTP Request: POST http://localhost:10003 "HTTP/1.1 200 OK"
INFO:google.adk.models.lite_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
You are an expert purchasing delegator that can delegate the user product inquiry and purchase request to the
appropriate seller remote agents.

Execution:
- For actionable tasks, you can use `send_task` to assign tasks to remote agents to perform.
- When the remote agent is repeatedly asking for user confirmation, assume that the remote agent doesn't have access to user's conversation context. 
    So improve the task description to include all the necessary information related to that agent
- Never ask user permission when you want to connect with remote agents. If you need to make connection with multiple remote agents, directly
    connect with them without asking user permission or asking user preference
- Always show the detailed response information from the seller agent and propagate 

Send Remote Agent Task Response: {'jsonrpc': '2.0', 'id': '270b3f2981d846e3a28767d889d3c878', 'result': {'id': '2a22fcfd-064b-4e2f-a955-0dbe5f5ee813', 'sessionId': '5fdeaeca-9a07-4693-a942-9fa4ca0398ad', 'status': {'state': 'completed', 'timestamp': '2025-07-29T09:50:50.837742'}, 'artifacts': [{'parts': [{'type': 'text', 'text': 'Your order has been created successfully. Order ID: e319c3dd-5baa-4e87-b30c-107f94ab0994. Order items: Classic Cheeseburger (1) - IDR 85,000, Spicy Chicken Burger (1) - IDR 80,000. Total: IDR 165,000.'}], 'index': 0}], 'history': []}}


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/home/pbaskara/trial/Jupyter/orig/tut_branhc/purchasing-concierge-intro-a2a-codelab-starter/.venv/lib/python3.12/site-packages/opentelemetry/trace/__init__.py", line 589, in use_span
    yield span
  File "/home/pbaskara/trial/Jupyter/orig/tut_branhc/purchasing-concierge-intro-a2a-codelab-starter/.venv/lib/python3.12/site-packages/opentelemetry/sdk/trace/__init__.py", line 1105, in start_as_current_span
    yield span
  File "/home/pbaskara/trial/Jupyter/orig/tut_branhc/purchasing-concierge-intro-a2a-codelab-starter/.venv/lib/python3.12/site-packages/google/adk/runners.py", lin